<a href="https://colab.research.google.com/github/Acabre21/machineLearning/blob/master/HW_5/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DISCLAIMER:** not all code is runnable since adding complete relevant examples would be too long. All the code used is from my previous assignments. 

### 1. General Concepts


**Artificial Intelligence** is simply the idea of computers exhibiting human-like intelligence but with the power of computation. Traditional AI gives the software a set of rules and input, which the machine uses to try to spit out an output. Machine learning instead will feed input and output, and the machine will try to spit out the rules. The rules are then used to predict outputs for new, unseen inputs. 

**Machine Learning** falls under the AI umbrella and is the process of training a piece of software to make useful predictions given a data set. 

**Deep Learning** is a subset of Machine learning and involves layers of algorithms which allows for more complex models and naturally, more complex data processing. 

Two common training methods are **supervised** and **unsupervised learning**. The idea behind supervised learning is to provide the software, data with corresponding labels, and have it become familiar with the data and its labels. Then, unlabeled data is fed to the sofware, and based off of the previous **training**, it will then be able to predict the labels for this new data. Unsupervised learning is training with no labels, which forces the software to find it's own patterns and therefore it's own labels for the data. 

The code below is an example of supervised learning. The machine is fed images which are passed through different layers which learn to identify key features. A convolutional neural network is prominently used in image classification. The **convolution** process is described below. A filter convolves over the input matrix to identify features and typically make the image smaller as well.This means the dot product of the filter and the extracted tile(a piece of the input matrix, the same size of the filter), is taken for every pixel in the the image. A **convolved feature** is the output, this is a part of the image that focuses on a specific feature such as a line corner. It is then sent to a pooling layer. The **pooling layer** makes the image smaller by picking out the most engaged pixels in the image and getting rid of the less engaged pixels. It uses a process similar to convolution as shown below and discussed in section 3. 

In [0]:
  # declaring and initializing the convolved feature
  # convolution loop
  out_mat = np.zeros((out_dim, out_dim))
  for i in range(out_dim):
      for j in range(out_dim):
        # applying the filter to each tile and outputting it to the matrix
          out_mat[i][j] = np.sum(input_mat[i:i+kernel_dim, j:j+kernel_dim]*kernel_mat)

  # applying the s by s window over the convolved feature and storing it in a queue to later store in an output matrix
  # pooling loop
  for i in range(0, input_dim, s):
    out_dim+=1
    for j in range(0, input_dim, s):
        # iterating through pools of size s and finding the max
        q.append(np.max(input_mat[i:i+s, j:j+s]))

### 2. Basic Concepts

Since there are so many different types of data (facial recognition, weather prediction, spam recognition...), different learning algorithms are used to fit specific needs. One such algorithm is **linear regression:** a supervised ML learning algorithm that looks at independent variables(x) and makes a prediction on a dependent variable(y). It does so by fitting a line to a labeled dataset, once it has the line of best fit, it can take new x's and predict their y. Another similar algorithm is called **logistic regression** which works better for predicting categorical variables as opposed to the continuous variables that linear regression is better suited for. This is because logistic regression produces a value between 0-1 using a sigmoid curve, that determines the probability of whether it will rain tomorrow, or if an email is spam, some categorical value. Linear regression produces continuous values such as salary, age, and weight using a line.  

During training, the model is bound to make many incorrect predictions. **Loss** is the quantitative measurement of how wrong the prediction is. **Accuracy** is the measure of how many correct predictions were made. The example below looks at functions that determine the loss and accuracy of a model. Typically, models have their own built-in functions but we implemented this one for HW3. 

In [0]:
def loss(data, labels, w, b, threshold):
  sum = 0
  for i in range(len(data)):
    z = w * data[i][0] + w * data[i][1] + b
    a = 1/ (1 + np.exp(-z))
    sum += labels[i] - a
  return sum/len(data)

  
def accuracy(data, labels, w, b, threshold):
  correct = 0

  for i in range(len(data)):
    z = w * data[i][0] + w * data[i][1] + b
    a = 1/ (1 + np.exp(-z))
    if a >= threshold:
      a = 1
    else:
      a = 0

    if labels[i]==a:
      correct+=1
  return correct/len(data)

Learning algorithms like the two discussed above, both use loss minimization algorithms to speed up the training process. One such algorithm is **Gradient Descent:** An algorithm that computes the parameter updates for loss minimization. This is done by using the gradient of the loss function to gradually get closer and closer to correct predictions until the desired values are found. The machine uses the gradient to descend to the answer. 

The code below is the logistic regression we implemented in HW3. As described above, the function uses Stochastic GD which uses the gradient of the loss function to predict the weight and the bias. The learning rate is the size of the jump in predictions.  

In [0]:
def logisticRegression(data, labels):
  epochs = len(data)
  lr = 0.01

  # initial value for weight w and bias b
  w = np.random.randn(1)
  b = np.zeros(1)

  # stochastic gradient descent, going through one label at a time
  for i in range(epochs):
    #activation function
    z = w * data[i][0] + w * data[i][1] + b
    # probability 
    a = 1/ (1 + np.exp(-z))

    # gradient of the cross-entropy loss function
    gradWeight = (a - labels[i]) * data[i][0] + (a - labels[i]) * data[i][1] 
    gradBias = a - labels[i]

    # finding the next weight and bias to test
    w = w -(lr * gradWeight)
    b = b -(lr * gradBias)
  return w,b

### 3. Building a Model

A convolutional neural network is a neural network with the addition of convolutional layers. CNN's are especially useful in the classification of images. The convulational layers detect features ranging in complexity such as line edges or dog species. Once the image makes it through the convolutional layers, it is processed by the classifier which will detect more specific features and then actually classify the image. 

The **convolutional layer** recieves as parameters: the size of the tiles being extracted and the depth (number of filters to apply). The dot product is then calculated with the extracted tiles and the filter, for every pixel in the image. Since filtering is most exhaustive part of the model, the ideal model uses an optimal number of filters. Filtering, results in an output feature map/convolved feature, a pixel matrix. A **ReLU** function is then applied to introduce nonlinearity into the model. Nonlinearity allows the machine to learn more complex data. After that, the resulting matrix is passed into a pooling layer.

A pooling layer takes an image and compresses it into a smaller matrix while still maintaining the key features of the image. This significantly lightens the computational load. A common pooling algorithm is **Max pooling.** The process is very similar to convolution, but instead of going over every single pixel, a filter(usually 2x2) moves by *s* pixels across the matrix until the whole image has been processed, where *s* is the stride. The filter can be thought of as a pool, and from the pool, the max is selected and placed in the compressed output matrix. This maintains the critical sections of the image and speeds up the whole process. The max of each pool is the most engaged pixel or in other words, the pixel that is likely more important to what the image is. 
Eventually the model processes the whole image and it is categorized. 

The code below shows the structure of a CNN being built. First comes the convolutional base Xception which identifies features with increasing complexity. It is comprised of several convolutional and pooling layers. This is followed by the classifier which is made up of two dense layers, one with 512 neurons, and one with 10. The image will be passed through the base and the classifier before finally being identified. 

In [0]:
import tensorflow as tf
%tensorflow_version 2.x
from keras.applications import Xception

conv_base = Xception(
    weights='imagenet', 
    include_top=False, 
    input_shape=(150, 150, 3))

# convolutional base followed by classifier
model = tf.keras.models.Sequential()
model.add(conv_base)

# relu introduces nonlinearity 
# softmax produces classification probabilities 
model.add(tf.keras.layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

### 4. Comping a Model

A model tries to minimize loss with the usage of **optimizers** and **learning rates**. An optimzer function like Stochastic Gradient Descent allows the model to reach faster, accurate predictions. It does so by taking the derivative(s) of the loss function with respect to the independent variable(s) and multiplies it by the learning rate to approach the desired values as shown in section 2. A **learning rate** is simply the number that is multiplied with the resulting gradient to find the next parameters to try. The learning rate is the size of the steps taken towards the minimized loss. If the learning isn't an appropriate size, the model may end up diverging or getting stuck in a local minimum. A learning rate has to be played around with before it is optimal. Typically a value between 0.0001 and 0.01. A learning rate is one of those hyper-parameters like which activation function to use, or which loss function to use, that may differ from machine to machine. It's something that needs to be tweaked when trying to build an optimal network. 

In this case, the model below recieves the categorical cross entropy function and the optimizer rms prop. **Categorical cross entropy** will determine how close each prediction is. **rms prop** uses the gradient of the categorical cross entropy function to lower the loss and take **learning rate** sized steps toward the right values. 

In [0]:
# categorical_crossentropy is used to determine how close the model's predictions are
# rmsprop dynamically updates the learning rate to minimize loss
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

### 5. Training a Model

One of the problems involved with training is the potential of over/under fitting. Overfitting is a situation where the model becomes very familiar and adept at working with the training data, but is so tuned in to that sample, that it doesn't preform nearly as well with a new set of data. This will result in low loss for the training data and high loss for new test data. This is a result of an overcomplicated model that is not generalized enough. That is why simplicity is key in forming a model. Underfitting is when a model can't recognize the training data which produces low accuracy and high loss. Underfitting is typically caused by the opposite of overfitting, the model isn't complex enough. Layers and neurons may be added to combat this. 

In this code, the model recives mini-batch supervised learning for 20 epochs. It'll make predictions on the testImages and have it verified by the testLabels. An epoch is the number of times the machine will go through all the samples. Instead of working with the whole data set at once, instead it'll work with a batch of 128 samples at a time. This slows down the overall training process but is done to enhance the training. Going through all the data at once can degrade the quality of the image recognition. 

In [0]:
e = 20
history = model.fit(trainImages, 
                      trainLabels, 
                      epochs=e, 
                      batch_size=128, 
                      validation_data=(testImages, testLabels))

### 6. Finetuning a pretrained Model

A neural network is a very complex machine of iteration. Trial and error, trial and error, over and over again until it can accurately classify information. **Finetuning** is the idea of taking a model that has already been trained for a task, and tweaking it to work with a similar but different task. This is much easier than building a model from scratch. This can be implemented with the addition and removal of layers into the old model. 

For example, if a model was trained to classify images into soccer or basketballs, it could be finetuned to recognize tennis balls as well. Perhaps the final classification layer would be restructured and then the model could now classify all three. 

Building a model involves so many different variables, and decisions, and training, and iteration, and the list goes on and on. Because of all this, it is common practice in the industry to finetune a machine instead building one. This is exactly what we did for HW4, the code below shows the addition of a dense layer to change the classifier for the CNN. Adding a new dense layer increases the potential for feature identification and could be used to recognize a broader set of data. 



In [0]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
# new Dense layer for recognizing new data
model.add(layers.Dense(256, activation='softmax'))
model.add(layers.Dense(1, activation='sigmoid'))

When adding a layer into a machine that's being finetuned, the base of the machine can be frozen. This means that the weights won't change and will not be updated during the training of the new layer. This is vital to maintaining and updating the functionality of the machine. In HW4 we first froze the base so that the previous training wouldn't be affected by the training of the new layer. Layers can also be removed if their feature identification is no longer needed in the new finetuned machine. 

The code below loops through the convolutional layers and unfreezes all the layers after conv2d_6. This means that all the layers before that maintain weights that we want to preserve. Layers after that are being tweaked to now function as a part of the finetuned machine. 

In [0]:
for layer in conv_base.layers:
  # unfreezing conv2d_6
  if layer.name == 'conv2d_6':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

Overall I learned a lot about the fundamentals about AI. It was fun to peel back the curtain on a lot of concepts that initially mystified me. I always heard about AI but never had a working understanding of how it actually worked. Leaving this class I can say that I have established a small base for understanding AI and have an appreciation for the complexity and work that goes behind it. Machine learning, Deep learning, Stochastic Gradient Descent, optimizers ... I now know all these concepts and their implementations. However, this class has merely scratched the surface, I'm very excited to keep learning about AI until one day they have achieved WORLD DOMINATION. Haha just kidding. Learning these concepts has established a good foundation for me to build on. I'm excited to see what the future holds for AI. 